# Kriging with external drift estimation on Jura

## Load Packages

In [ ]:
import numpy as np
import pandas as pd
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import minigst as mg


## Load the data set and the prediction grid


We start by loading the data and the prediction grid

In [ ]:
jura_all, grid, _ = mg.data("Jura")

Change the names of the Land Use and Rock in order to be consistent with their names on the grid

In [ ]:
## Replace landuse interger code by name
landuse_codes=[1,2,3,4]
landuse_names=["Forest","Pasture","Meadow","Tillage"]
jura_all["Landuse"]=jura_all["Landuse"].replace(landuse_codes,landuse_names)

## Replace rock type integer code by name
rock_codes=[1,2,3,4,5]
rock_names=["Argovian","Kimmeridgian","Sequanian","Portlandian","Quaternary"]
jura_all["Rock"]=jura_all["Rock"].replace(rock_codes,rock_names)

Build the predictor variables corresponding to Rock with one-hot encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# drop the first column for each feature
enc = OneHotEncoder(handle_unknown='ignore',drop='first')
enc.fit(jura_all[["Rock"]])
newnames = ["Rock_K","Rock_P","Rock_Q","Rock_S"]
rock_indic_jura = pd.DataFrame(enc.transform(jura_all[["Rock"]]).toarray(),columns = newnames)
jura_all = pd.concat([jura_all,rock_indic_jura],axis=1)
rock_indic_grid = pd.DataFrame(enc.transform(grid[["Rock"]]).toarray(),columns = newnames)
grid = pd.concat([grid,rock_indic_grid],axis=1)

Separate the data set in two sets : the training set and the validation set.
For the project and the Kaggle competition, you should use the full data set for 
the training.
You will submit your prediction on Kaggle for a set of locations on which you will 
only know the locations and the factors of Land Use and Rock at these locations.

In [ ]:
ntot = jura_all.shape[0]
ntrain = 200
nval = ntot - ntrain
np.random.seed(123454)
indtrain = np.random.choice(ntot,ntrain,replace=False).astype(int)
indval = np.setdiff1d(np.arange(ntot),indtrain)

jura =jura_all.loc[indtrain,:]

#val contains the values to predict. For the project, these values will be on Kaggle
#(for other locations) and you won't know them
#You will have the locations and covariables at the unknown locations by the following command :
val_loc =jura_all.loc[indval,['Xloc','Yloc'] + newnames]
val=jura_all.loc[indval,['Co']]

## Gstlearn objects

First, we create a gstlearn database containing the data points, and assign the appropriate locators to the variables.

In [ ]:
## Create Db
db_jura=mg.df_to_db(jura, coord_names= ["Xloc","Yloc"])
db_jura.display()

We also create a gstlearn *Grid Database* containing the target grid for the prediction.

In [ ]:
### Load grid data into  point database
db_grid=mg.df_to_dbgrid(grid, coord_names= ["Xloc","Yloc"])

## Add selection to points with defined Rock type
mg.add_sel(db_grid, ~np.isnan(db_grid["Rock_K"]) ) 

Finally, we create a Db containing the validation locations and the value of Cobalt concentrations at those locations.

In [ ]:
## Create Db
db_val=mg.df_to_db(val_loc, coord_names= ["Xloc","Yloc"])

## Add Co values
db_val["Co"]=val

db_val.display()


## Variography of the residuals

Compute the variogram of the residuals

In [ ]:
## Create experimental variogram :
## With - 30 lags separated by a distance 0.1 (meaning that we compute the variogram at lags h=0.1*i for i=0,...,30),
## and consider a tolerance τ=50% on the distance

#Vario of the residuals
varioResiduals = mg.vario_exp(db_jura, vname= "Co", ext_drift= ["Rock_*"], nlag = 30, dlag = 0.1, toldis = 0.5)


#Vario of the raw variable for comparison
varioRaw = mg.vario_exp(db_jura, vname= "Co", nlag = 30, dlag = 0.1, toldis = 0.5)


## Plot
ax = gp.varmod(varioRaw,showPairs=False,label = "Raw")
ax = gp.varmod(varioResiduals,showPairs=False,color = "r",label = "Residual")
ax = plt.legend()

Model fitting

In [ ]:
fitmodRaw = mg.model_fit(varioRaw, struct = ["NUGGET","EXPONENTIAL"])
fitmodResiduals = mg.model_fit(varioResiduals, struct = ["NUGGET","EXPONENTIAL"])

## Plot
gp.varmod(varioResiduals, fitmodResiduals)
gp.varmod(varioRaw, fitmodRaw,color="r")
plt.show()

## Kriging with external Drift

The *kriging* function is called to perform the kriging with external drift

In [ ]:
## Remove variables starting with a given prefix (-> Results from previous runs)
mg.del_var_from_db(db_grid ,["KED*"])
mg.del_var_from_db(db_grid, ["OK*"])
           
## Compute kriging
err = mg.minikriging(db_jura,db_grid, vname = "Co", ext_drift= ["Rock_*"], model = fitmodResiduals, prefix = "KED") 

## Compute ordinary kriging for comparison
err = mg.minikriging(db_jura,db_grid, vname = "Co", model = fitmodRaw, prefix = "OK") 

## Display database
db_grid.display()

In [ ]:
## Plot prediction
im = gp.plot(db_grid, "KED*estim")
gp.plot(db_jura, c='black')
plt.title("Kriging with external Drift prediction")
plt.colorbar(im)
plt.show()

## Plot kriging standard-deviation
im = gp.plot(db_grid, "KED*stdev")
gp.plot(db_jura, c='black')
plt.title("Kriging with external Drift standard-deviation")
plt.colorbar(im)
plt.show()

We can ask for the regression coefficients using *regression*.

In [ ]:
mg.regression(db_jura, "Co",  ext_drift = ["Rock_*"], model = fitmodRaw)

Computing the KED prediction at the validation locations and the resulting RMSE can then be done as follows

In [ ]:
## Remove variables starting with a given prefix (-> Results from previous runs)
mg.del_var_from_db(db_val,["OK*"])
mg.del_var_from_db(db_val,["KED*"])

## Compute kriging
err = mg.minikriging(db_jura,db_val, vname = "Co", ext_drift= ["Rock_*"], model = fitmodResiduals, prefix = "KED") 

## Compute ordinary kriging for comparison
err = mg.minikriging(db_jura,db_val, vname = "Co", model = fitmodRaw, prefix = "OK") 

## Compute RMSE
rmse_KED=np.mean((db_val["Co"]-db_val["KED*estim"])**2)**0.5
rmse_OK=np.mean((db_val["Co"]-db_val["OK*estim"])**2)**0.5

print("Ordinary Kriging RMSE",rmse_OK)
print("KED RMSE",rmse_KED)

## Questions

1. Improve the model by adding other explanatory variables (e.g. Landuse or the interactions Rock*Landuse).
2. Estimate the model parameters by maximum Likelihood.
3. Define and adjust a multivariate model with drift. Compute the associated predictions.